In [1]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 20160510: both versions of BkAvgAst created (nohist & ultcert)
###           used Tickers from earlier version, then supplemented with missing Tickers if aeTotAst>800bn
### 20170407: updated for 2016-Dec updated FailedBank list and 2017-Mar institutions2.csv file
### 20180827: updated for 2018-Jun FailedBank list and 2018-Aug institutions2.csv file

import numpy as np
import pandas as pd
from datetime import datetime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [2]:
rk = pd.read_csv("BkAvgAstPR_FailDate_201812.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2018-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2018-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2018-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2018-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.482006e+09  1.456936e+09  5.203546e+08  6.834806e+08   
1 2001-03-31  1.270022e+09  1.273816e+09  6.647306e+08  8.105992e+08   
2 2001-03-31  1.063243e+09  1.060575e+09  5.082855e+08  2.856345e+08   
3 2001-03-31  8.926864e+08  8.810598e+08  5.379178e+08  6.205972e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.541728e+08  9.376534e+08        NaT  9.202367   9.202367  5.670701   
1  1.054239e+08  9.160231e+08 2009-01-16  7.886074  17.088441  7.244076   
2  4.414540e+08  7.270885e+08 2008-11-23  6.602099  23.690540  5.539174   
3  6.737132e+07  6.879685e+08        NaT  5.543047  29.233587  5.862100   
4  2.469325e+07  2.951088e+08        NaT  2.742359  31.975946  2.774973   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.670701  7.448409   7.448409  
1  12.914777  8.833718  16.282127  
2  18.453951  3.112777  19.394904  
3  24.316051  6.763120  26.158025  
4  27.091024  2.946925  29.104950

In [3]:
len(rk) # 10775

10775

In [4]:
bhc = pd.read_csv("INSTITUTIONS2_201808.csv", encoding='latin-1') # Warning: This is latest available file as of 2019-April.
bhc.head()

STNAME  CERT  DOCKET  ACTIVE              ADDRESS      ASSET BKCLASS  \
0  Connecticut     4     0.0     0.0       22 Main Street     48,570      NM   
1  Connecticut     6     0.0     0.0  81 West Main Street    624,655      NM   
2        Maine     8     0.0     0.0      One City Center  1,699,404      SM   
3        Maine     9  9781.0     0.0       66 Main Street    539,169      SM   
4        Maine    10     0.0     0.0  8 Washington Street     55,692      SM   

   CHANGEC1  CHANGEC2  CHANGEC3  ...  \
0     223.0       0.0       0.0  ...   
1     223.0     999.0       0.0  ...   
2     223.0       0.0       0.0  ...   
3     223.0       0.0       0.0  ...   
4     223.0       0.0       0.0  ...   

                                       CBSA  CBSA_NO  \
0  Hartford-West Hartford-East Hartford, CT  25540.0   
1                     New Haven-Milford, CT  35300.0   
2               Portland-South Portland, ME  38860.0   
3                                       NaN      0.0   
4               Portland-South Portland, ME  38860.0   

                            CBSA_METRO_NAME CBSA_METRO CBSA_METRO_FLG  \
0  Hartford-West Hartford-East Hartford, CT    25540.0            1.0   
1                     New Haven-Milford, CT    35300.0            1.0   
2               Portland-South Portland, ME    38860.0            1.0   
3                                       NaN        0.0            0.0   
4               Portland-South Portland, ME    38860.0            1.0   

  CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0            0.0       NaN          0.0          0.0 NaN  
1            0.0       NaN          0.0          0.0 NaN  
2            0.0       NaN          0.0          0.0 NaN  
3            0.0       NaN          0.0          0.0 NaN  
4            0.0       NaN          0.0          0.0 NaN  

[5 rows x 133 columns]

In [5]:
bhc.replace(to_replace='orporated', value='', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='inancial', value='in', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=' Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=', The', value='', inplace=True, regex=True)

In [6]:
len(bhc) # 27607

27607

In [7]:
bhc.head(2)

STNAME  CERT  DOCKET  ACTIVE              ADDRESS    ASSET BKCLASS  \
0  Connecticut     4     0.0     0.0       22 Main Street   48,570      NM   
1  Connecticut     6     0.0     0.0  81 West Main Street  624,655      NM   

   CHANGEC1  CHANGEC2  CHANGEC3  ...  \
0     223.0       0.0       0.0  ...   
1     223.0     999.0       0.0  ...   

                                       CBSA  CBSA_NO  \
0  Hartford-West Hartford-East Hartford, CT  25540.0   
1                     New Haven-Milford, CT  35300.0   

                            CBSA_METRO_NAME CBSA_METRO CBSA_METRO_FLG  \
0  Hartford-West Hartford-East Hartford, CT    25540.0            1.0   
1                     New Haven-Milford, CT    35300.0            1.0   

  CBSA_MICRO_FLG  CBSA_DIV  CBSA_DIV_NO CBSA_DIV_FLG  CB  
0            0.0       NaN          0.0          0.0 NaN  
1            0.0       NaN          0.0          0.0 NaN  

[2 rows x 133 columns]

In [8]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
#bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
bhc2.rename(index=str, columns={"CERT":"Cert", "FED_RSSD":"IDRSSD", "NAME":"Name", "STALP":"ST", "BKCLASS":"BkClass", \
            "OFFICES":"Offices", "NAMEHCR":"Parent", "RSSDHCR":"ParRSSD", "STALPHCR":"ParST", "ULTCERT":"UltCert"}, inplace=True)
len(bhc2) # 27607

27607

In [9]:
bhc2.head(2)

Cert    IDRSSD                                    Name  ST BkClass  \
0     4  573401.0  The Southington Bank and Trust Company  CT      NM   
1     6  148304.0              Colonial Bank of Waterbury  CT      NM   

   Offices Parent  ParRSSD ParST  UltCert  
0      4.0    NaN      NaN   NaN   3510.0  
1     34.0    NaN      NaN   NaN   3510.0

In [10]:
bhc2.columns

Index([u'Cert', u'IDRSSD', u'Name', u'ST', u'BkClass', u'Offices', u'Parent',
       u'ParRSSD', u'ParST', u'UltCert'],
      dtype='object')

In [11]:
bhc2[bhc2['Parent']>' '][0:10]

Cert    IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301.0  State Street Bank and Trust Company  MA      SM      1.0   
15    35  749635.0                           AuburnBank  AL      SM      8.0   
19    39  430036.0            Robertson Banking Company  AL      NM      7.0   
21    41  432432.0                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937.0            Merchants Bank of Alabama  AL      NM      5.0   
26    47  390336.0               Traders & Farmers Bank  AL      NM      7.0   
29    50  470032.0                Community Spirit Bank  AL      NM      5.0   
30    51  382537.0                   The Bank of Vernon  AL      NM      3.0   
31    52  293437.0        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537.0         The Exchange Bank of Alabama  AL      NM      5.0   

                               Parent    ParRSSD ParST  UltCert  
5                   State Street Corp  1111435.0    MA     14.0  
15        Auburn National Bancorp Inc  1129533.0    AL     35.0  
19                            Rbc Inc  2526755.0    AL     39.0  
21       Phenix-Girard Bancshares Inc  2013246.0    AL     41.0  
25         Merchants Fin Services Inc  1082955.0    AL     46.0  
26   Traders & Farmers Bancshares Inc  3467305.0    AL     47.0  
29  Independent Bancshares IncESOPlan  3587294.0    AL     50.0  
30        First Vernon Bancshares Inc  1139998.0    AL     51.0  
31   Citizens Bancorp Of Winfield Inc  1833616.0    AL     52.0  
33                       Gadsden Corp  1079067.0    AL     54.0

In [12]:
#bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]  ## should be null list

In [14]:
#bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

In [15]:
## extract ticker info from prior listing: (rkhn was expanded output from br001f_NewBHCtickers code)
rkh = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201806.csv", index_col=0)
#rkhn = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201612n.csv", index_col=0)

#len(rkh), len(rkhn) # 10762
len(rkh) # 10772

10772

In [16]:
len(rkh[rkh['Ticker']>'']) # 3102 

3102

In [17]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkh2 = rkh[rkh['ParRSSD']>0][['Rank','ParRSSD','ParST','Ticker']]
len(rkh2) # 8765

8765

In [18]:
rkh2.tail(2)

Rank    ParRSSD ParST Ticker
10768  10769  1094314.0    MO  CBCYB
10771  10772  3447567.0    TX    NaN

In [19]:
rkh3 = rkh2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_index(by='Rank',inplace=True)
len(rkh3) # 397

397

In [20]:
rkh3.head()

Rank ParST
ParRSSD   Ticker            
1039502.0 JPM        1    NY
1073757.0 BAC        2    NC
1951350.0 C          3    NY
1120754.0 WFC        4    CA
1119794.0 USB        6    MN

In [21]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN

In [22]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABTX          1
ABYB          1
ACFC          1
ACNB          1

In [23]:
rkh5[rkh5['ParRSSD']>1] # finds tickers with more than one ParRSSD code:

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [24]:
rkh[rkh['Ticker']=="BHWB"].iloc[:,0:12] # check on Ticker that was misidentified in earlier runs (should be 2 banks only)

Rank  IDRSSD       xAsOf           xName xState    xCert       nAsOf  \
1741  1742   27548  2018-06-30  BLACKHAWK BANK     WI  14078.0  2001-03-31   
7717  7718  388931  2003-09-30   FIRST BANK BC     IL  16551.0  2001-03-31   

     FailDate BkClass  Offices                 Parent    ParRSSD  
1741      NaN      NM     10.0  Blackhawk Bancorp Inc  1491913.0  
7717      NaN      NM      4.0  Blackhawk Bancorp Inc  1491913.0

In [25]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert    IDRSSD                                    Name  ST BkClass  \
0     4  573401.0  The Southington Bank and Trust Company  CT      NM   
1     6  148304.0              Colonial Bank of Waterbury  CT      NM   
2     8  422406.0                     Fleet Bank of Maine  ME      SM   
3     9  563907.0                     Union Trust Company  ME      SM   
4    10  112109.0               Northeast Bank of Sanford  ME      SM   

   Offices Parent  ParRSSD ParST  UltCert Ticker  
0      4.0    NaN      NaN   NaN   3510.0    NaN  
1     34.0    NaN      NaN   NaN   3510.0    NaN  
2     55.0    NaN      NaN   NaN   3510.0    NaN  
3     13.0    NaN      NaN   NaN   4255.0    NaN  
4      5.0    NaN      NaN   NaN   3510.0    NaN

In [26]:
bhc3[bhc3['Cert']==3510]

Cert    IDRSSD                                   Name  ST BkClass  \
2233  3510  480228.0  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4488.0  Bank Of America Corp  1073757.0    NC   3510.0    BAC

In [27]:
bhc2[bhc2['Cert']==3510]

Cert    IDRSSD                                   Name  ST BkClass  \
2233  3510  480228.0  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4488.0  Bank Of America Corp  1073757.0    NC   3510.0

In [28]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2018-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2018-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...   pr_eTLcs    pr_eDD  \
0  1.482006e+09  1.456936e+09  5.203546e+08  ...   5.670701  7.448409   
1  1.270022e+09  1.273816e+09  6.647306e+08  ...  12.914777  8.833718   

    pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  \
0   7.448409   628.0        N   5202.0  Jpmorgan Chase & Co.  1039502.0   
1  16.282127  3510.0        N   4488.0  Bank Of America Corp  1073757.0   

   ParST  UltCert  
0     NY    628.0  
1     NC   3510.0  

[2 rows x 27 columns]

In [29]:
rkh4.head(2)

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC

In [30]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2018-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2018-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...    pr_eDD   pr_eDDcs    Cert  \
0  1.482006e+09  1.456936e+09  5.203546e+08  ...  7.448409   7.448409   628.0   
1  1.270022e+09  1.273816e+09  6.647306e+08  ...  8.833718  16.282127  3510.0   

  BkClass  Offices                Parent    ParRSSD  ParST  UltCert  Ticker  
0       N   5202.0  Jpmorgan Chase & Co.  1039502.0     NY    628.0     JPM  
1       N   4488.0  Bank Of America Corp  1073757.0     NC   3510.0     BAC  

[2 rows x 28 columns]

In [31]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [32]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2018-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2018-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices  ...       aeTotLn        aeDDep        aeFDep  \
0        NaT       N   5202.0  ...  5.203546e+08  6.834806e+08  2.541728e+08   
1 2009-01-16       N   4488.0  ...  6.647306e+08  8.105992e+08  1.054239e+08   

       aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD   pr_eDDcs  
0  9.376534e+08  9.202367   9.202367  5.670701   5.670701  7.448409   7.448409  
1  9.160231e+08  7.886074  17.088441  7.244076  12.914777  8.833718  16.282127  

[2 rows x 27 columns]

In [33]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist201812.csv")

In [34]:
len(rk4) # 10775

10775

In [35]:
## find bhcs with totast above 700MM and no ticker:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm) # 207

207

In [36]:
rkm.head()

Rank  IDRSSD      xAsOf                         xName xState    xCert  \
144   145  527954 2018-12-31  FIRST NATIONAL BANK OF OMAHA     NE   5452.0   
149   150  311845 2018-12-31                   ARVEST BANK     AR   8728.0   
174   175  137915 2018-12-31                 EMIGRANT BANK     NY  12054.0   
175   176  877369 2018-12-31                  RABOBANK, NA     CA  23364.0   
179   180  249612 2018-12-31        APPLE BANK FOR SAVINGS     NY  16068.0   

         nAsOf FailDate BkClass  Offices  ...       aeTotLn        aeDDep  \
144 2001-03-31      NaT       N    128.0  ...  7.929635e+06  9.278700e+06   
149 2001-03-31      NaT      SM    303.0  ...  6.967262e+06  9.254315e+06   
174 2001-03-31      NaT      SB      5.0  ...  5.102184e+06  6.206396e+06   
175 2001-03-31      NaT       N    107.0  ...  6.173227e+06  6.182788e+06   
179 2001-03-31      NaT      SB     80.0  ...  4.461945e+06  7.545158e+06   

          aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
144     0.000000  9.278700e+06  0.072214  73.947656  0.086415  69.970384   
149  3196.444444  9.257512e+06  0.068111  74.299808  0.075928  70.389249   
174     0.000000  6.206396e+06  0.055301  75.817454  0.055602  72.047791   
175     0.000000  6.182788e+06  0.054854  75.872308  0.067274  72.115065   
179     0.000000  7.545158e+06  0.054243  76.090636  0.048625  72.360670   

       pr_eDD   pr_eDDcs  
144  0.101117  74.095708  
149  0.100851  74.483585  
174  0.067636  76.399603  
175  0.067379  76.466982  
179  0.082225  76.705038  

[5 rows x 27 columns]

In [37]:
rkm.to_csv('rk_missingtickers201812.csv')

In [38]:
##############################################################################

In [39]:
## start up here after filling in missing tickers

In [40]:
## filled in many missing tickers

In [41]:
rkm2 = pd.read_csv('rk_missingtickers201812a.csv',index_col=0)
rkm2.head()

Rank  IDRSSD       xAsOf                         xName xState  xCert  \
144   145  527954  12/31/2018  FIRST NATIONAL BANK OF OMAHA     NE   5452   
149   150  311845  12/31/2018                   ARVEST BANK     AR   8728   
174   175  137915  12/31/2018                 EMIGRANT BANK     NY  12054   
175   176  877369  12/31/2018                  RABOBANK, NA     CA  23364   
179   180  249612  12/31/2018        APPLE BANK FOR SAVINGS     NY  16068   

         nAsOf FailDate BkClass  Offices  ...      aeTotLn       aeDDep  \
144  3/31/2001      NaN       N      128  ...  7929634.847  9278699.750   
149  3/31/2001      NaN      SM      303  ...  6967261.861  9254315.097   
174  3/31/2001      NaN      SB        5  ...  5102183.542  6206395.778   
175  3/31/2001      NaN       N      107  ...  6173226.778  6182788.167   
179  3/31/2001      NaN      SB       80  ...  4461945.125  7545157.694   

          aeFDep     aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
144     0.000000  9278699.750  0.072214  73.947656  0.086415  69.970384   
149  3196.444444  9257511.542  0.068111  74.299808  0.075928  70.389249   
174     0.000000  6206395.778  0.055301  75.817454  0.055602  72.047791   
175     0.000000  6182788.167  0.054854  75.872308  0.067274  72.115065   
179     0.000000  7545157.694  0.054243  76.090636  0.048625  72.360670   

       pr_eDD   pr_eDDcs  
144  0.101117  74.095708  
149  0.100851  74.483586  
174  0.067636  76.399603  
175  0.067379  76.466982  
179  0.082225  76.705038  

[5 rows x 27 columns]

In [42]:
rkm3 = rkm2[rkm2['Ticker']>' '][['ParRSSD','Ticker']]
rkm3.head()

ParRSSD Ticker
288  3814208    LBC
332  2609975   OCFC
549  1885307   OBNK
728  1204627     BY
942  3530786   MBIN

In [43]:
rkm3p = set(rkm3['ParRSSD'])
len(rkm3), len(rkm3p) # 11, 11

(11, 11)

In [44]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName  \
288                              LUTHER BURBANK SAVINGS   
332                                 OCEANFIRST BANK, NA   
549                                         ORIGIN BANK   
704                           ATLANTIC CAPITAL BANK, NA   
728                                         BYLINE BANK   
942                           MERCHANTS BANK OF INDIANA   
985   INDUSTRIAL AND COMMERCIAL BANK OF CHINA (USA), NA   
991                                         FSGBANK, NA   
1112                                        EQUITY BANK   
1187                                    MALVERN BANK NA   
1234                                      SOUTHERN BANK   
1237                                      OLD LINE BANK   
9320                 FARMERS-MERCHANTS BANK OF ILLINOIS   

                               Parent Ticker  
288               Luther Burbank Corp    NaN  
332              Oceanfirst Fin Corp.    NaN  
549                Origin Bancorp Inc    NaN  
704   Atlantic Capital Bancshares Inc    NaN  
728                Byline Bancorp Inc    NaN  
942                 Merchants Bancorp    NaN  
985             China Investment Corp    NaN  
991   Atlantic Capital Bancshares Inc    NaN  
1112            Equity Bancshares Inc    NaN  
1187             Malvern Bancorp, Inc    NaN  
1234    Southern Missouri Bancorp Inc    NaN  
1237          Old Line Bancshares Inc    NaN  
9320                Merchants Bancorp    NaN

In [45]:
rk4c = rk4.copy()

In [46]:
tdct = dict(zip(rkm3['ParRSSD'],rkm3['Ticker']))
tdct

{1204627L: 'BY',
 1885307L: 'OBNK',
 2609975L: 'OCFC',
 3180547L: 'EQBK',
 3200221L: 'OLBK',
 3266227L: 'SMBC',
 3530786L: 'MBIN',
 3555686L: 'ACBI',
 3805279L: 'MLVF',
 3814208L: 'LBC',
 4097978L: 'CIC'}

In [47]:
## update missing Tickers if it exists:
for i in np.arange(len(rk4)):
  rs = rk4.ix[i]['ParRSSD']
  try:
    rk4.loc[i,'Ticker'] = tdct[rs]
  except:
    pass

In [48]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName  \
288                              LUTHER BURBANK SAVINGS   
332                                 OCEANFIRST BANK, NA   
549                                         ORIGIN BANK   
704                           ATLANTIC CAPITAL BANK, NA   
728                                         BYLINE BANK   
942                           MERCHANTS BANK OF INDIANA   
985   INDUSTRIAL AND COMMERCIAL BANK OF CHINA (USA), NA   
991                                         FSGBANK, NA   
1112                                        EQUITY BANK   
1187                                    MALVERN BANK NA   
1234                                      SOUTHERN BANK   
1237                                      OLD LINE BANK   
9320                 FARMERS-MERCHANTS BANK OF ILLINOIS   

                               Parent Ticker  
288               Luther Burbank Corp    LBC  
332              Oceanfirst Fin Corp.   OCFC  
549                Origin Bancorp Inc   OBNK  
704   Atlantic Capital Bancshares Inc   ACBI  
728                Byline Bancorp Inc     BY  
942                 Merchants Bancorp   MBIN  
985             China Investment Corp    CIC  
991   Atlantic Capital Bancshares Inc   ACBI  
1112            Equity Bancshares Inc   EQBK  
1187             Malvern Bancorp, Inc   MLVF  
1234    Southern Missouri Bancorp Inc   SMBC  
1237          Old Line Bancshares Inc   OLBK  
9320                Merchants Bancorp   MBIN

In [49]:
len(rk4[rk4['Ticker']>' ']),len(rk4c[rk4c['Ticker']>' ']) # 518, 509

(522, 509)

In [50]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist201812.csv")

In [51]:
## process ultcert version:

In [52]:
rk5 = rk4.copy()

In [53]:
ult = set(rk5['UltCert'])
len(ult) # 5873

5873

In [54]:
bhcc = rk5[rk5['xCert'].isin(ult)][['xCert','Parent','Ticker','ParRSSD','ParST']]
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [55]:
bhcc.dropna(subset=['Parent'],inplace=True)
len(bhcc) # 4303

4303

In [56]:
len(bhcc[bhcc['Ticker']>'']) # 511

511

In [57]:
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [58]:
bhcc.groupby('xCert').filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

xCert                           Parent Ticker    ParRSSD ParST
38     16571.0                 Bank Of Montreal    BMO  1231333.0     0
196     4999.0              Cadence Bancorp Llc   CADE  4037349.0    TX
256    58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
467    26342.0             Royal Bank Of Canada     RY  1232497.0     0
486    35385.0        United Community Fin Corp   UCFC  3831250.0    OH
671     4999.0              Cadence Bancorp Llc   CADE  4037349.0    TX
704    35525.0  Atlantic Capital Bancshares Inc   ACBI  3555686.0    GA
991    35525.0  Atlantic Capital Bancshares Inc   ACBI  3555686.0    GA
1078    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
1188   57197.0                      Bnccorp Inc   BNCC  1248162.0    ND
1604    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
1711   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
1777   16571.0                 Bank Of Montreal    BMO  1231333.0     0
1779      89.0   White River Bancshares Company    NaN  3306589.0    AR
1998   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
2183   58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
2317   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
2657   26342.0             Royal Bank Of Canada     RY  1232497.0     0
2946   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4054   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
4220   35385.0        United Community Fin Corp   UCFC  3831250.0    OH
4337   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4473   26575.0            South Shore Mutual HC    NaN  3919738.0    MA
4539   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
4639   17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
4981    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
5960    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
6799   17399.0                   First Fin Corp    NaN  1067514.0    KS
7091      89.0   White River Bancshares Company    NaN  3306589.0    AR
7096   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
7432   13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
7490   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
7583   15432.0               Uptown Bancorp Inc    NaN  1206797.0    SD
7751    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
8764   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
9110   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
9666    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
9785   17399.0                   First Fin Corp    NaN  1067514.0    KS
10163  15432.0               Uptown Bancorp Inc    NaN  1206797.0    SD
10195  13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
10232  17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
10517  57197.0                      Bnccorp Inc   BNCC  1248162.0    ND

In [59]:
bhcc[bhcc['xCert']==17399]

xCert          Parent Ticker    ParRSSD ParST
6799  17399.0  First Fin Corp    NaN  1067514.0    KS
9785  17399.0  First Fin Corp    NaN  1067514.0    KS

In [60]:
len(bhcc) # 4303

4303

In [61]:
bhcc2 = bhcc.drop_duplicates(subset=['xCert','Parent','Ticker','ParRSSD'],keep=False)
len(bhcc2) # 4261

4261

In [62]:
bhcc2.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [63]:
bhcc2.set_index(['xCert'],drop=True,inplace=True)
bhcc2.head()

Parent Ticker    ParRSSD ParST
xCert                                                
628.0    Jpmorgan Chase & Co.    JPM  1039502.0    NY
3510.0   Bank Of America Corp    BAC  1073757.0    NC
7213.0          Citigroup Inc      C  1951350.0    NY
3511.0  Wells Fargo & Company    WFC  1120754.0    CA
6548.0           U.S. Bancorp    USB  1119794.0    MN

In [64]:
bhcc2.groupby(level=0).filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

Empty DataFrame
Columns: [Parent, Ticker, ParRSSD, ParST]
Index: []

In [65]:
dcrt = bhcc2[['Parent','Ticker','ParRSSD','ParST']].to_dict()
dcrt['Parent'][628]

u'Jpmorgan Chase & Co.'

In [66]:
##### LOOK AT THIS!!! IMPORTANT CONDITIONAL PROCESSING FOR MISSING VALUES!!!!  ## improved 5/9/2016
##########################################################################
j = 0
rk5c = rk5.copy()
for i in np.arange(len(rk5c)):
  if (pd.isnull(rk5c.ix[i]['Parent']))|(rk5c.ix[i]['Parent']==' '):
    xc = rk5c.ix[i]['UltCert']
    try:
      rk5c.loc[i,'Parent'] = dcrt['Parent'][xc]
      rk5c.loc[i,'Ticker'] = dcrt['Ticker'][xc]
      rk5c.loc[i,'ParRSSD'] = dcrt['ParRSSD'][xc]
      rk5c.loc[i,'ParST'] = dcrt['ParST'][xc]
      j = j+1
    except:
      pass

In [67]:
len(rk5c[rk5c['Ticker']>'']) # 3170

3170

In [68]:
j # 4401

4401

In [69]:
len(rk5c) # 10775

10775

In [70]:
rk5c.to_csv("BkAvgAstPR_FailDate_BHC2_ult201812.csv")

In [71]:
# I use ZION to show how many historic banks and mergers got collapsed into one bank charter

In [72]:
rk4[rk4['Ticker']=='ZION']  ## current active affiliate (1 bank list)

Rank  IDRSSD      xAsOf                     xName xState   xCert  \
78    79  276579 2018-12-31  ZIONS BANCORPORATION, NA     UT  2270.0   

        nAsOf FailDate BkClass  Offices  ...       aeTotLn        aeDDep  \
78 2001-03-31      NaT       N    444.0  ...  1.671393e+07  1.820146e+07   

           aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
78  850974.708333  1.905244e+07  0.148998  67.164222  0.182144  62.556748   

      pr_eDD   pr_eDDcs  
78  0.198355  66.107981  

[1 rows x 27 columns]

In [73]:
rk5c[rk5c['Ticker']=='ZION']  ## all acquired & consolidated banks (23 banks)

Rank   IDRSSD      xAsOf                                 xName xState  \
78      79   276579 2018-12-31              ZIONS BANCORPORATION, NA     UT   
153    154   837260 2015-09-30               CALIFORNIA BANK & TRUST     CA   
160    161   676656 2015-09-30                        AMEGY BANK, NA     TX   
307    308  1004368 2015-09-30              NATIONAL BANK OF ARIZONA     AZ   
334    335   456960 2015-09-30                     NEVADA STATE BANK     NV   
436    437   933957 2015-09-30              VECTRA BANK COLORADO, NA     NM   
715    716   807665 2009-06-30                     VINEYARD BANK, NA     CA   
1097  1098   458263 2006-12-31                  STOCKMEN'S BANK, THE     AZ   
1144  1145  1198641 2015-09-30  COMMERCE BANK OF WASHINGTON, NA, THE     WA   
1212  1213  2479310 2008-06-30                     SILVER STATE BANK     NV   
1629  1630   720465 2008-12-31                         ALLIANCE BANK     CA   
1707  1708   618656 2004-09-30                            KLEIN BANK     TX   
2742  2743   105950 2001-09-30          MINNEQUA BANK OF PUEBLO, THE     CO   
3204  3205   576158 2003-06-30                             MAXIMBANK     TX   
4114  4115   309664 2006-06-30                           RANCHO BANK     CA   
4206  4207   521550 2003-12-31                        LONE STAR BANK     TX   
4681  4682  2068134 2009-03-31            GREAT BASIN BANK OF NEVADA     NV   
6214  6215   626754 2001-03-31              LEAGUE CITY BANK & TRUST     TX   
6411  6412  3161993 2008-03-31                           CHOICE BANK     AZ   
6700  6701   687269 2007-06-30        INTERCONTINENTAL NATIONAL BANK     TX   
6730  6731   456670 2002-09-30                   FRONTIER STATE BANK     AZ   
8427  8428  3374186 2015-03-31          COMMERCE BANK OF OREGON, THE     OR   
9905  9906  2907448 2003-06-30               SOUTHLAND BUSINESS BANK     CA   

        xCert      nAsOf   FailDate BkClass  Offices  ...       aeTotLn  \
78     2270.0 2001-03-31        NaT       N    444.0  ...  1.671393e+07   
153   20852.0 2001-03-31        NaT      NM     99.0  ...  7.675023e+06   
160   24107.0 2001-03-31        NaT       N     81.0  ...  6.722988e+06   
307   20626.0 2001-03-31        NaT       N     66.0  ...  3.325315e+06   
334   18113.0 2001-03-31        NaT      NM     50.0  ...  2.461972e+06   
436    2993.0 2001-03-31        NaT       N     37.0  ...  1.905816e+06   
715   23556.0 2001-03-31 2009-07-17       N     17.0  ...  1.101171e+06   
1097  23182.0 2001-03-31        NaT      NM     43.0  ...  4.397506e+05   
1144  27298.0 2001-03-31        NaT       N      2.0  ...  4.893643e+05   
1212  34194.0 2001-03-31 2008-09-05      NM     18.0  ...  6.252840e+05   
1629  23124.0 2001-03-31 2009-02-06      NM      5.0  ...  4.423914e+05   
1707  22866.0 2001-03-31        NaT      NM     27.0  ...  1.706073e+05   
2742   1773.0 2001-03-31        NaT      NM      5.0  ...  2.003063e+05   
3204   9017.0 2001-03-31        NaT      NM      9.0  ...  8.677390e+04   
4114  22934.0 2001-03-31        NaT      NM      4.0  ...  1.179183e+05   
4206  22304.0 2001-03-31        NaT      NM      6.0  ...  1.375284e+05   
4681  33824.0 2001-03-31 2009-04-17      NM      5.0  ...  1.067878e+05   
6214  17577.0 2001-03-31        NaT      NM      4.0  ...  4.341800e+04   
6411  57469.0 2003-03-31        NaT      NM      3.0  ...  8.134071e+04   
6700  19112.0 2001-03-31        NaT       N      3.0  ...  6.690315e+04   
6730  24227.0 2001-03-31        NaT      NM      7.0  ...  6.683700e+04   
8427  58223.0 2005-12-31        NaT      NM      1.0  ...  4.561513e+04   
9905  35287.0 2001-03-31        NaT      NM      1.0  ...  1.870840e+04   

            aeDDep         aeFDep      aeTotDep    pr_eTA   pr_eTAcs  \
78    1.820146e+07  850974.708333  1.905244e+07  0.148998  67.164222   
153   8.519186e+06   32194.661017  8.551380e+06  0.064035  74.565117   
160   7.001789e+06  646260.186441  7.648049e+06  0.061251  75.000920   
307   3.496657e+06  